In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from collections import namedtuple

In [3]:
#import tensorflow.compat.v1 as tf
#tf.disable_v2_behavior()

In [4]:


# Setting the Hyperparameters
HyperParams = namedtuple('HyperParams', ['num_steps',
                                         'max_seq_len',
                                         'input_seq_width',
                                         'output_seq_width',
                                         'rnn_size',
                                         'batch_size',
                                         'grad_clip',
                                         'num_mixture',
                                         'learning_rate',
                                         'decay_rate',
                                         'min_learning_rate',
                                         'use_layer_norm',
                                         'use_recurrent_dropout',
                                         'recurrent_dropout_prob',
                                         'use_input_dropout',
                                         'input_dropout_prob',
                                         'use_output_dropout',
                                         'output_dropout_prob',
                                         'is_training',
                                         ])

# Making a function the returns all the values of the default hyperparameters
def get_default_hps():
    return HyperParams(num_steps=2000,
                       max_seq_len=1000,
                       input_seq_width=35,
                       output_seq_width=32,
                       rnn_size=256,
                       batch_size=100,
                       grad_clip= 1.0,
                       num_mixture=5,
                       learning_rate=0.001,
                       decay_rate=1.0,
                       min_learning_rate=0.00001,
                       use_layer_norm=0,
                       use_recurrent_dropout=0,
                       recurrent_dropout_prob=0.90,
                       use_input_dropout=0,
                       input_dropout_prob=0.90,
                       use_output_dropout=0,
                       output_dropout_prob=0.90,
                       is_training=1,
                       )

# Getting these default hyperparameters
hps = get_default_hps()

In [5]:
print(hps)

HyperParams(num_steps=2000, max_seq_len=1000, input_seq_width=35, output_seq_width=32, rnn_size=256, batch_size=100, grad_clip=1.0, num_mixture=5, learning_rate=0.001, decay_rate=1.0, min_learning_rate=1e-05, use_layer_norm=0, use_recurrent_dropout=0, recurrent_dropout_prob=0.9, use_input_dropout=0, input_dropout_prob=0.9, use_output_dropout=0, output_dropout_prob=0.9, is_training=1)


In [6]:
# Building the RNN
KMIX = hps.num_mixture
INWIDTH = hps.input_seq_width
OUTWIDTH = hps.output_seq_width
LENGTH = hps.max_seq_len
if hps.is_training:
    global_step = tf.Variable(0, name='global_step', trainable=False)

In [7]:
cell_fn = tf.keras.layers.LSTMCell

In [8]:
use_recurrent_dropout = False if hps.use_recurrent_dropout == 0 else True
use_input_dropout = False if hps.use_input_dropout == 0 else True
use_output_dropout = False if hps.use_output_dropout == 0 else True
use_layer_norm = False if hps.use_layer_norm == 0 else True

In [9]:
cell_fn

keras.layers.rnn.lstm.LSTMCell

In [10]:
if use_recurrent_dropout:
    cell = cell_fn(hps.rnn_size, dropout_keep_prob=hps.recurrent_dropout_prob) # layer_norm is ommitted since it is not supported by the LSTM wrapper
else:
    cell = cell_fn(hps.rnn_size)

In [11]:
if use_input_dropout:
    cell = tf.keras.layers.Dropout(cell, input_keep_prob=hps.input_dropout_prob)
if use_output_dropout:
    cell = tf.keras.layers.Dropout(cell, output_keep_prob=hps.output_dropout_prob)

In [12]:
sequence_lengths = LENGTH
tf.compat.v1.disable_eager_execution()
# tf.config.experimental_run_functions_eagerly(True)
input_x = tf.compat.v1.placeholder(dtype=tf.float32, shape=[hps.batch_size, hps.max_seq_len, INWIDTH], name='input_x')
output_x = tf.compat.v1.placeholder(dtype=tf.float32, shape=[hps.batch_size, hps.max_seq_len, OUTWIDTH], name='output_x')
actual_input = input_x
rnn = tf.keras.layers.RNN(cell)
initial_state = [tf.zeros([hps.batch_size, size]) for size in rnn.cell.state_size]
#initial_state = rnn.zero_state(batch_size=hps.batch_size, dtype=tf.float32)
NOUT = OUTWIDTH * KMIX * 3

In [13]:
with tf.compat.v1.variable_scope('RNN'):
        output_w = tf.compat.v1.get_variable("output_w", [hps.rnn_size, NOUT]) # Output weights
        output_b = tf.compat.v1.get_variable("output_b", [NOUT]) # Output biases

In [14]:
output, last_state = tf.compat.v1.nn.dynamic_rnn(cell=cell,
                                                inputs=actual_input,
                                                initial_state=initial_state,
                                                dtype=tf.float32,
                                                swap_memory=True,
                                                scope='RNN')

Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API


In [15]:
output

<tf.Tensor 'RNN_1/transpose_1:0' shape=(100, 1000, 256) dtype=float32>